In [191]:
import numpy as np
import scipy
import pylab
import plotly.graph_objects as go

import pylab
import matplotlib
from matplotlib.pyplot import figure
from mpl_toolkits import mplot3d
import numpy as np

from qiskit import Aer
from qiskit.opflow import X, Z, Y, I
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.circuit.library import TwoLocal, QAOAAnsatz

from qiskit.algorithms.optimizers import ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, NELDER_MEAD, \
                                            NFT, POWELL, SLSQP, SPSA, TNC


#Imports Functions for Building Hamiltonains
%run Utils.ipynb
np.set_printoptions(formatter={'float': '{:0.3f}'.format})

In [206]:
m = 1

x_res = 10
x_min, x_max = 0.1, 2

n_max = 14
n_res = (n_max-4)//2 + 1

x_val, qubits = np.meshgrid(1 / (np.linspace(x_min, x_max, x_res) ** 2), np.linspace(4, n_max ,n_res))
f0 = np.zeros([n_res, x_res], dtype=float)
f1 = np.zeros([n_res, x_res], dtype=float)
f2 = np.zeros([n_res, x_res], dtype=float)

for i in range(n_res):
    for j in range(x_res):
        f0[i,j], f1[i,j], f2[i,j] = findDimensionlessEigen(x_val[i,j], qubits[i,j], m)
        print(f'{((i * x_res) + j + 1) / (x_res * n_res) * 100 :3.2f}%   Qubit Count: {4 + i * 2}', end='\r', flush = True)
print('\nDone!')

100.00%   Qubit Count: 14
Done!


In [207]:
fx = lambda x: 1/np.sqrt(x)
fy = lambda y: 1/y

# Ground State Convergence

In [208]:
x_axis, y_axis = fx(x_val), fy(qubits)

fig = go.Figure(data=[go.Surface(x=x_axis, y=y_axis, z=f0, opacity=0.7, showscale=False, colorscale='Viridis'), 
                      go.Surface(x=x_axis, y=y_axis, z=np.full([n_res, x_res], -1/np.pi), opacity=0.3, showscale=False, colorscale='Viridis')])

fig.update_layout(title='Groundstate Convergence',
                   autosize=False,
                  width=1000, height=700,
                 scene = dict(
                    xaxis_title='1/sqrt(x)',
                    yaxis_title='N',
                    zaxis_title='Energy'))
fig.show()

# First Excited State Convergence

In [161]:
fig = go.Figure(data=[go.Surface(x=x_axis, y=y_axis, z=f1, opacity=0.7, colorscale='Viridis'), 
                      go.Surface(x=x_axis, y=y_axis, z=np.full([n_res, x_res], 2/np.sqrt(np.pi)), opacity=0.3, showscale=False, colorscale='Viridis')])

fig.update_layout(title='First Excited State Convergence',
                   autosize=False,
                  width=1000, height=700,
                 scene = dict(
                    xaxis_title='1/sqrt(x)',
                    yaxis_title='N',
                    zaxis_title='Energy'))
fig.show()

# Second Excited State Convergence

In [169]:
fig = go.Figure(data=[go.Surface(x=x_axis, y=y_axis, z=f2, opacity=0.7, colorscale='Viridis'),
                      go.Surface(x=x_axis, y=y_axis, z=np.full([n_res, x_res], 1/np.sqrt(np.pi)), opacity=0.3, showscale=False, colorscale='Viridis')])

fig.update_layout(title='Excited State Convergence',
                   autosize=False,
                  width=1000, height=700,
                 scene = dict(
                    xaxis_title='1/sqrt(x)',
                    yaxis_title='N',
                    zaxis_title='Energy'))
fig.show()

In [186]:
print(np.amin(f2)/.37)

2.0768571688264017


# Extrapolation

In [198]:
def gen_extrapolated_data(x_data, y_data, z_data, extrap_range, extrap_axis, poly_size = 5, get_intercept = False):
    
    assert extrap_axis == 'y' or extrap_axis =='x'
    
    x_data, y_data = x_data[0], y_data.T[0]
    
    if extrap_axis == 'y':
        x_range, y_range = len(x_data), len(extrap_range)
        x_data_extrap, y_data_extrap = np.meshgrid(x_data, extrap_range)
        dep_axis = np.array(y_data)
    else:
        x_range, y_range = len(y_data), len(extrap_range)
        x_data_extrap, y_data_extrap = np.meshgrid(extrap_range, y_data)
        dep_axis = np.array(x_data)

    poly_data = [np.zeros([x_range, y_range], dtype=float) for _ in range(poly_size)]
    
    if extrap_axis == 'y':
        z_data = z_data.T
        
    poly_intercepts = np.zeros([poly_size, len(z_data)])
    
    for i in range(len(z_data)):
        indep_axis = np.array(z_data[i])
        intercept = []
        for p in range(poly_size):
            coeffs = np.polyfit(dep_axis, indep_axis, p + 1)
            poly_intercepts[p, i] = coeffs[-1]
            poly_data[p][i] = np.poly1d(coeffs)(extrap_range)
            
    if extrap_axis == 'y':
        poly_data = [poly.T for poly in poly_data]
    
    if get_intercept:
        return x_data_extrap, y_data_extrap, poly_data, poly_intercepts
    return x_data_extrap, y_data_extrap, poly_data

In [209]:
x_ext = np.linspace(0, x_max , 10)
n_ext = np.linspace(0, .25 , 8)

extrap_x_f0, extrap_y_f0, extrap_poly_f0 = gen_extrapolated_data(fx(x_val), fy(qubits), f0, n_ext, 'y')
extrap_x_f0, extrap_y_f0, extrap_poly_f0 = gen_extrapolated_data(extrap_x_f0, extrap_y_f0, extrap_poly_f0[4], x_ext, 'x')

extrap_f0 = extrap_poly_f0[4]

extrap_x_f1, extrap_y_f1, extrap_polys_f1 = gen_extrapolated_data(fx(x_val), fy(qubits), f1, n_ext, 'y')
extrap_x_f1, extrap_y_f1, extrap_polys_f1 = gen_extrapolated_data(extrap_x_f1, extrap_y_f1, extrap_polys_f1[3], x_ext, 'x' )

extrap_f1 = extrap_polys_f1[3]

extrap_x_f2, extrap_y_f2, extrap_polys_f2 = gen_extrapolated_data(fx(x_val), fy(qubits), f2, n_ext, 'y')
extrap_x_f2, extrap_y_f2, extrap_polys_f2 = gen_extrapolated_data(extrap_x_f2, extrap_y_f2, extrap_polys_f2[3], x_ext, 'x' )

extrap_f2 = extrap_polys_f2[1]

In [210]:
fig = go.Figure(data=[go.Surface(x=fx(x_val), y=fy(qubits), z=f0, opacity=0.7, showscale=False, colorscale='Viridis'), 
                      go.Surface(x=extrap_x_f0, y=extrap_y_f0, z=np.full([len(extrap_x_f0), len(extrap_y_f0.T)], -1/np.pi), opacity=0.3, showscale=False, colorscale='Viridis'),
                      go.Surface(x=extrap_x_f0, y=extrap_y_f0, z=extrap_f0, opacity=0.2, showscale=False, colorscale='Viridis')])

fig.update_layout(title='Extrapolated w0 Convergence',
                   autosize=False,
                  width=1000, height=700,
                 scene = dict(
                    xaxis_title='1/sqrt(x)',
                    yaxis_title='1/N',
                    zaxis_title='Energy'))
fig.show()

print(f"{extrap_E0[0,0]} - Extrapolated Convergence")
print(f"{-1/np.pi} - Actual Convergence")

-0.3175190425713299 - Extrapolated Convergence
-0.3183098861837907 - Actual Convergence


In [166]:
fig = go.Figure(data=[go.Surface(x=fx(x_val), y=fy(qubits), z=f1, opacity=0.7, showscale=False, colorscale='Viridis'), 
                      go.Surface(x=extrap_x_f1, y=extrap_y_f1, z=np.full([len(extrap_x_f1), len(extrap_y_f1.T)], 1/np.sqrt(np.pi)), opacity=0.3, showscale=False, colorscale='Viridis'),
                      go.Surface(x=extrap_x_f1, y=extrap_y_f1, z=extrap_f1, opacity=0.2, showscale=False, colorscale='Viridis')])

fig.update_layout(title='Extrapolated w1 Convergence',
                   autosize=False,
                  width=1000, height=700,
                 scene = dict(
                    xaxis_title='1/sqrt(x)',
                    yaxis_title='1/N',
                    zaxis_title='Energy'))
fig.show()
print(f"{extrap_M_[0,0]} - Extrapolated Convergence")
print(f"{1/np.sqrt(np.pi)} - Actual Convergence")

0.18393401164870407 - Extrapolated Convergence
0.5641895835477563 - Actual Convergence


In [167]:
fig = go.Figure(data=[go.Surface(x=fx(x_val), y=fy(qubits), z=f2, opacity=0.7, showscale=False, colorscale='Viridis'), 
                      go.Surface(x=extrap_x_f2, y=extrap_y_f2, z=np.full([len(extrap_x_f2), len(extrap_y_f1.T)], 1/np.sqrt(np.pi)), opacity=0.3, showscale=False, colorscale='Viridis'),
                      go.Surface(x=extrap_x_f2, y=extrap_y_f2, z=extrap_f2, opacity=0.2, showscale=False, colorscale='Viridis')])

fig.update_layout(title='Extrapolated w2 Convergence',
                   autosize=False,
                  width=1000, height=700,
                 scene = dict(
                    xaxis_title='1/sqrt(x)',
                    yaxis_title='1/N',
                    zaxis_title='Energy'))
fig.show()
print(f"{extrap_M_[0,0]} - Extrapolated Convergence")
print(f"{1/np.sqrt(np.pi)} - Actual Convergence")

0.18393401164870407 - Extrapolated Convergence
0.5641895835477563 - Actual Convergence
